# Iterative Prisoner's Dilemma

## Description

## Project's aims

### Strategies

* **Nice guy**: always cooperates (the function's output is always $u_D$)
* **Bad guy**: always defects 
* **Mainly nice**: randomly defects $k\%$ of the times and cooperates $100-k\%$, $k<50$
* **Mainly bad**: randomly defects $k\%$ of the times and cooperates $100-k\%$, $k>50$
* **Tit-for-tat**: starts by cooperating, then repeats what the opponent has done in the previous move
* **Random**:cooperates or defects randomly
* **Grim**: starts by cooperating, but if the opponent defects one time, it'll always defect
* **Forgiving Tit-Tat**: starts by cooperating, then repeats what the opponent has done in the previous move, but defects only if the opponent defects for two consecutive rounds
* **Suspicious Tit-Tat**: starts by defecting, then repeats what the opponent has done in the previous move
* **Pavlov**: cooperates if in the previous round has done the same move of the opponent, otherwise defects 
* **Reactive Nice**: in the first round cooperates $50\%$ of the times, then cooperates $70\%$ of the times and defects $30\%$
* **Reactive Bad**: in the first round cooperates $50\%$ of the times, then cooperates $30\%$ of the times and defects $70\%$
* **Hard Joss**: plays like Tit fot Tat, but cooperates only with probability $0.9$
* **Soft Joss**: plays like Tit fot Tat, but defects only with probability $0.9$

## Project's structure

This project is structured as follow:
* **Point one: Fight 1 vs 1**:
* ****:
* ** **:

### Libraries and imported files

We have implemented some libraries and, in order to simplify and streamline the main project, we have created strategies.py, update_func.py and it_pris_dil_func.py, which we'll discuss later.

In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import it_pris_dil_func as pris_dil

## Point 1: IPD between two players

In this first part we want to implement a iterated prisoner's dilemma implementing two given strategies.

### Code regarding the strategies implemented

We report above the code inside **strategies.py**

In [ ]:
import numpy.random as npr

uc, ud = [1,0], [0,1]
k=0.3

def nice_guy(it,u,v,u2):
    return uc

def bad_guy(it,u,v,u2):
    return ud

def mainly_nice(it,u,v,u2):
    a = npr.rand()
    if a > k: return uc
    else: return ud
    
def mainly_bad(it,u,v,u2):
    a = npr.rand()
    if a <= k: return uc
    else: return ud
    
def tit_tat(it,u,v,u2):
    if it==2: return uc
    else: return u
    
def random(it,u,v,u2):
    a = npr.rand()
    if a < 0.5: return uc
    else: return ud
    
def pavlov(it,u,v,u2):
    if it==2:
        return uc
    else:
        if v == ud:
            return ud
        else:
            if u == ud:
                return ud
            else: return uc

def f_tit_tat(it,u,v,u2):
    if it <= 3: return uc
    else:
        if u == ud and u2 == ud:
            return ud
        else: return uc

As input variables we have:
* **it**: current iteration(round) of the 1vs1 fight
* **u**: opponent's move in the previous round
* **u2**: opponent's move two rounds before
* **v**: player's move in the previous round

## 1vs1 Fight function

Here is reported the function to calculate the fight 1vs1 which consists in a game of the standard prisoner dilemma between two given strategies, iterated for a chosen number of times.

In [ ]:
def fight(f,g,probf=0,probg=0,N=None,graph=False):

    if N == None: N = 100
            
    R, S, T, P = 3, 0, 5, 1
    M = np.array([[R,S],[T,P]])

    p1, p2 = [-1,-1], [-1,-1]
    for i in range(2,N+2):
        if probf == 0 and probg == 0:
            p1.append(strat[f](i,p2[i-1],p1[i-1],p2[i-2]))
            p2.append(strat[g](i,p1[i-1],p2[i-1],p1[i-2]))
        else:
            p1.append(mutation(f,probf,i,p2[i-1],p1[i-1],p2[i-2]))
            p2.append(mutation(g,probg,i,p1[i-1],p2[i-1],p1[i-2]))

    p1 = np.array(p1[2:]).T
    p2 = np.array(p2[2:]).T

    result_1 = np.cumsum([np.dot(p1[:,i].T,np.dot(M,p2[:,i])) for i in range(N)])
    result_2 = np.cumsum([np.dot(p2[:,i].T,np.dot(M,p1[:,i])) for i in range(N)])

    if graph == True:
        plt.xlabel('iteration')
        plt.ylabel('points')
        plt.plot(result_1, label=f)
        plt.plot(result_2, label=g)
        plt.legend()

    return [result_1[-1], result_2[-1]]

For simplicity in this first point we use only the standard strategies, represented in a list called **s**.
We use this function in a nested for loop in order to create the **result matrix**, containing the score of all the fights' combinations.

In [ ]:
s = ['nice','bad','m_nice','m_bad','tit_tat']

result = np.zeros((len(s),len(s)))
for i in range(len(s)):
    for j in range(i,len(s)):
        p1, p2 = pris_dil.fight(s[i],s[j])
        result[i,j] = p1
        result[j,i] = p2

df = pd.DataFrame(result,dtype=int,columns=s,index=s)

#Results representation
def colorcell(x):
    data = x.copy()
    data.loc[:,:] = ''
    for i in range(5):
        for j in range(5):
            if result[i,j] < result[j,i]: data.iloc[i,j] = 'color : red'
            elif result[i,j] > result[j,i]: data.iloc[i,j] = 'color : green'
            else: data.iloc[i,j] = 'color : lightblue'
    return data

sum = np.sum(result, axis=1)
df['result'] = sum
df = df.astype({'result': int})
df = df.style.apply(colorcell,axis=None)

display(df)